In [52]:
import numpy as np
import pandas as pd

from fastai.imports import *
from fastai.tabular import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from pathlib import Path

## Config

In [2]:
data_pth = './avocado-prices.zip'

## Data

In [103]:
df_raw = (pd.read_csv(data_pth, low_memory=False, parse_dates=['Date'])
        .drop('Unnamed: 0', axis=1)).sort_values('Date')

In [104]:
df_raw.head().T

,11569,9593,10009,1819,9333
Date,2015-01-04 00:00:00,2015-01-04 00:00:00,2015-01-04 00:00:00,2015-01-04 00:00:00,2015-01-04 00:00:00
AveragePrice,1.75,1.49,1.68,1.52,1.64
Total Volume,27365.9,17723.2,2896.72,54956.8,1505.12
4046,9307.34,1189.35,161.68,3013.04,1.27
4225,3844.81,15628.3,206.96,35456.9,1129.5
4770,615.28,0,0,1561.7,0
Total Bags,13598.5,905.55,2528.08,14925.2,374.35
Small Bags,13061.1,905.55,2528.08,11264.8,186.67
Large Bags,537.36,0,0,3660.38,187.68
XLarge Bags,0,0,0,0,0


## Pre-processing

In [105]:
add_datepart(df_raw, 'Date')

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
11569,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.00,organic,...,4,6,4,False,False,False,False,False,False,1420329600
9593,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.00,organic,...,4,6,4,False,False,False,False,False,False,1420329600
10009,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.00,organic,...,4,6,4,False,False,False,False,False,False,1420329600
1819,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.00,conventional,...,4,6,4,False,False,False,False,False,False,1420329600
9333,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.00,organic,...,4,6,4,False,False,False,False,False,False,1420329600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8574,1.36,908202.13,142681.06,463136.28,174975.75,127409.04,103579.41,22467.04,1362.59,conventional,...,25,6,84,False,False,False,False,False,False,1521936000
9018,0.70,9010588.32,3999735.71,966589.50,30130.82,4014132.29,3398569.92,546409.74,69152.63,conventional,...,25,6,84,False,False,False,False,False,False,1521936000
18141,1.42,163496.70,29253.30,5080.04,0.00,129163.36,109052.26,20111.10,0.00,organic,...,25,6,84,False,False,False,False,False,False,1521936000
17673,1.70,190257.38,29644.09,70982.10,0.00,89631.19,89424.11,207.08,0.00,organic,...,25,6,84,False,False,False,False,False,False,1521936000


In [106]:
procs = [Categorify]

In [107]:
valid_idx = range(len(df_raw)-2000, len(df_raw))

In [108]:
dep_var = 'AveragePrice'
cat_names = ['type', 'region']
cont_names = [x for x in df_raw.columns if x not in (cat_names)]

In [109]:
for proc in procs:
    transformation = proc(cat_names, cont_names)
    transformation.apply_train(df_raw)

In [110]:
for cname in cat_names:
    df_raw[cname] = df_raw[cname].cat.codes

In [111]:
df_raw.AveragePrice = np.log(df_raw.AveragePrice)

In [112]:
df_raw = df_raw.reset_index(drop=True)

In [113]:
df_raw = pd.get_dummies(df_raw, columns=cat_names)

In [114]:
df_raw.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year,...,region_44,region_45,region_46,region_47,region_48,region_49,region_50,region_51,region_52,region_53
0,0.559616,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,2015,...,0,0,1,0,0,0,0,0,0,0
1,0.398776,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,2015,...,0,0,0,0,0,0,0,0,0,0
2,0.518794,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,2015,...,0,0,0,0,0,0,0,0,0,0
3,0.418710,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,2015,...,0,0,0,0,0,0,0,0,0,0
4,0.494696,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,2015,...,0,0,0,0,0,0,0,0,0,0


In [115]:
tmp_path = Path('./tmp')

In [116]:
tmp_path.mkdir(exist_ok=True)

In [117]:
df_raw.to_feather(tmp_path/'raw')

In [118]:
df_raw = pd.read_feather(tmp_path/'raw')

In [119]:
df_raw

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year,...,region_44,region_45,region_46,region_47,region_48,region_49,region_50,region_51,region_52,region_53
0,0.559616,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.00,2015,...,0,0,1,0,0,0,0,0,0,0
1,0.398776,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.00,2015,...,0,0,0,0,0,0,0,0,0,0
2,0.518794,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.00,2015,...,0,0,0,0,0,0,0,0,0,0
3,0.418710,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.00,2015,...,0,0,0,0,0,0,0,0,0,0
4,0.494696,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.00,2015,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,0.307485,908202.13,142681.06,463136.28,174975.75,127409.04,103579.41,22467.04,1362.59,2018,...,0,0,0,0,0,0,0,0,0,0
18245,-0.356675,9010588.32,3999735.71,966589.50,30130.82,4014132.29,3398569.92,546409.74,69152.63,2018,...,0,1,0,0,0,0,0,0,0,0
18246,0.350657,163496.70,29253.30,5080.04,0.00,129163.36,109052.26,20111.10,0.00,2018,...,0,1,0,0,0,0,0,0,0,0
18247,0.530628,190257.38,29644.09,70982.10,0.00,89631.19,89424.11,207.08,0.00,2018,...,0,0,0,0,0,0,0,0,0,0


In [120]:
df, y = (df_raw.drop('AveragePrice', axis=1), df_raw['AveragePrice'])

## Modelling

In [121]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

n_valid = 2000
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [122]:
mdl = RandomForestRegressor(n_jobs=-1)

In [123]:
mdl.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [124]:
mdl.score(X_valid, y_valid)

0.48743739005736664

In [125]:
print('Train RMSE: ', np.sqrt(mean_squared_error(mdl.predict(X_train), y_train)))

Train RMSE:  0.02879015030440607


In [128]:
print('Test RMSE: ', np.sqrt(mean_squared_error(np.exp(mdl.predict(X_valid)), np.exp(y_valid))))

Test RMSE:  0.23496839599035763
